In [1]:
import joblib

In [4]:
encoding = joblib.load("../src/encoder.pkl")
scaling = joblib.load("../src/scaler.pkl")

In [5]:
features_dict = joblib.load("../src/features_config.pkl")

In [6]:
encoding

ColumnTransformer(transformers=[('cat', OneHotEncoder(drop='first'),
                                 ['EmploymentType', 'MaritalStatus',
                                  'LoanPurpose']),
                                ('bin',
                                 OrdinalEncoder(categories=[['No', 'Yes'],
                                                            ['No', 'Yes'],
                                                            ['No', 'Yes']]),
                                 ['HasCoSigner', 'HasMortgage',
                                  'HasDependents']),
                                ('ord',
                                 OrdinalEncoder(categories=[['High School',
                                                             "Bachelor's",
                                                             "Master's",
                                                             'PhD']]),
                                 ['Education']),
                                ('num', 'passthrough',
                                 ['Age', 'Income', 'LoanAmount', 'CreditScore',
                                  'MonthsEmployed', 'NumCreditLines',
                                  'InterestRate', 'LoanTerm', 'DTIRatio'])])

In [7]:
features_dict

{'binary_features': ['HasCoSigner', 'HasMortgage', 'HasDependents'],
 'binary_order': [['No', 'Yes'], ['No', 'Yes'], ['No', 'Yes']],
 'numeric_features': ['Age',
  'Income',
  'LoanAmount',
  'CreditScore',
  'MonthsEmployed',
  'NumCreditLines',
  'InterestRate',
  'LoanTerm',
  'DTIRatio'],
 'categorical_features': ['EmploymentType', 'MaritalStatus', 'LoanPurpose'],
 'ordinal_feature': ['Education'],
 'education_order': [['High School', "Bachelor's", "Master's", 'PhD']]}